
# Introduction to Aperture Photometry

This notebook is a basic introduction to aperture photometry and will demonstrate how to perform aperture photometry in python using the `photutils` package. In order to get the most out of this tutorial, you are encouraged to clone this Jupyter Notebook and experiment with the various code examples on your own. Questions, comments, and suggestions to the author are welcome.

1. [What is Photometry (In General)](#what-is-photometry)
1. [The Point Spread Function](#the-point-spread-function)
1. [What is Aperture Photometry](#what-is-aperture-photometry)
1. [Reading FITS Files](#reading-fits-files)
1. [Coordinates in an Image](#coordinates-in-an-image)
1. [Performing Photometry](#performing-photometry)

## What is Photometry (In General) <a class="anchor" id="what-is-photometry"></a>

Unlike spectra, which classify the energy output of a single object per wavelength, photometric observations are images taken of multiple objects over a range of wavelengths. In many ways photometric images are similar to pictures taken with a black and white camera. You can think of the image as a collection of pixels, each of which has some numeric value that describes how much light reached that particular point of the camera's sensor.

Photometric images can be taken over a variety of wavelength ranges. These wavelength ranges are referred to as ***bands***, and have standardized letter names. For example, H-band goes from 1.5 to 1.8 micrometers. To observe in a particular band, a telescope observer will use a filter that only lets light within the band pass through the telescope. After light has passed through the filter, the optics in the telescope guide the light onto a CCD sensor.

As mentioned before, this sensor is much the same as those found in a common digital camera, and is broken up into a grid of pixels. When a photon hits a pixel, it releases an electron. By counting the number of electrons in each pixel, we can determine the number of photons that landed at that part of the sensor. This value is typically recorded either in units of total photon counts or as photon counts per second. However, sometimes images are converted into other units, like ergs or nanomaggies. Be careful of the units, don't assume!

It is important to note that in a photometric image, pixels don't represent color values. Instead they represent a number of photons (possibly per second). Color can be added to the image after the fact, but in this notebook, we are only concerned with the numerical data.


## The Point Spread Function

Before talking about aperture photometry, we need to understand the ***point spread function***, also called the **PSF**.

Objects tend to become "spread out" and larger on a photometric image. This is caused by a combination of influences including imperfections in the telescope optics and various effects within the CCD. A sensible question to ask is how much a point source - such as a distant star - is spread out on an image. The answer to this question is the PSF. In fact, an apt definition for the point spread function is *"a function that describes how an imaging system represents a point source"*.

Shown below is an example of a Gaussian PSF. Remember that the tail ends of a Gaussian never actually reach zero, but instead go on forever. Take a moment to think about how this effects a photometric image (before I answer the question below).


-- insert image here -- Remember to insert this! 

Because the tail ends of the PSF go on forever, the light from any object is spread out infinitely across an image. Even though most of the light from a star is focused around a single point in an image, it is impossible to draw a circle around that star on an image and say "this is all of the light from that star". No matter how big we make that circle, there will always be light that falls outside of it on the image.


## What is Aperture Photometry <a class="anchor" id="what-is-aperture-photometry"></a>

Aperture photometry is a process where we determine the energy output of an object by analyzing photometric images. Pretend you are observing an object with a telescope and imagine a sphere surrounding the object you’re looking at. Also, imagine that this sphere has a radius that is the same as your distance from the object, so that you are on the surface of this sphere. The collection area of your telescope only covers a small portion of the surface area of the sphere. If we assume that the object emits photons evenly in all directions, we can then determine the total photon emissions of the object as

$\ \ \ \ Total\ Photon\ Counts = \frac{Observed\ Counts\ from\ Object}{Collection\ Area} * Total\ Area\ of\ Sphere = \frac{Counts\ Observed\ from\ Object}{Collection\ Area} * \pi r^2$

Where $r$ is the radius of the imaginary sphere, or equivalently, your distance from the object.

If we know how far away we are from an object, then the only information missing from the above equation is the photon counts we observed from an object. In order to find this value we use photometric images. In essence, we begin by drawing an aperture (e.g. a circle) around the object in an image. We then count the number of photons that fell within the aperture, also known as the ***aperture sum***.

It is very important to remember that the aperture sum is not the same as the total observed photon counts. Remember that because of the PSF, we can never create an aperture that contains the entire object. Instead we settle on creating an aperture that contains most of the object in the image. But we have to be careful. Too small of an aperture means you might not be including enough of the object you’re interested in. However, too large an aperture means your including more background noise and possibly even another object. In general, the aperture size you use will depend on the type of object you want (examples of different objects with different apertures?) to look at and the PSF of the images themselves.


## Reading FITS Files <a class="anchor" id="reading-fits-files"></a>

The Flexible Image Transport System (FITS) is a file format that is commonly used in astronomy to store and share image data. A FITS file can be thought of as a collection of one or more tables called Header Data Units (HDUs). The `astropy` package makes reading these tables extremely easy.


In [ ]:
from astropy.io import fits
hdu_list = fits.open("example.fits") # Creates a list of the HDUs in the file
hdu_list.info()


To make things simple, our example file only has two HDUs, but depending on the file you read there may be several. In most cases we are usually interested in the first, or "primary" HDU.

The HDU's two parts are the actual data and a header, which stores useful information. The data is stored as an array, where each element in that array represents the value of a corresponding pixel. The header stores information about the image itself. If you are unsure about the units of the pixels in an image, the header is a good place to check. The same goes for information like the date the image was taken, or the image's total exposure time. Here are a few examples:

In [ ]:
from matplotlib import pyplot as plt

primary_hdu = hdu_list[0]
header = primary_hdu.header
data = primary_hdu.data

print("These are the first 10 entries in the header:\n")
print(repr(header[:10]))

# You can access each entry in the header individually.
print("\nThe value of 'BITPIX' from the header is:", header['BITPIX'])

print("\nHere is a nice plot of the image:\n")
plt.imshow(primary_hdu.data, cmap='gray')
plt.colorbar()
plt.show()

## Coordinates in an image <a class="anchor" id="coordinates-in-an-image"></a>

The position of objects in an astronomical image are typically represented in either pixel coordinates or in celestial coordinates. Converting between these unit systems requires knowing the World Coordinate System (WCS) for a particular image. This information is stored within the header of the FITS file itself, usually in terms of the following three values:

1. 'CRPIX' : The pixel coordinates of a reference pixel
1. 'CRVAL' : The the real world coordinates of the referance pixel
1. 'CD' : The increment in real world coordinates between pixels

The `astropy.coordinates.SkyCoord` module has a lot of useful tools for converting between various coordinate systems, but we won't talk about them all here. Instead we show a few short examples of how to specify coordinates. Notice that no matter how we specify the coordinates, `SkyCoord` internally stores the coordinates in units of degrees.

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u

hdu_list = fits.open("example.fits")
primary_hdu = hdu_list[0]

# This retrieves the WCS information from the image
image_wcs = WCS(primary_hdu)


# Create a coordinate object from units of degrees
coord_from_degrees = SkyCoord(85, -2.5, unit=('deg'))
print('A SkyCord object created with inputs ra = 85 deg, dec = -2.5 deg :')
print(coord_from_degrees, '\n')


# Create a coordinate in celestial units
coord_from_hourangle = SkyCoord("1:12:43.2", "+1:12:43", unit=u.hourangle)
print('A SkyCord object created with hourangle inputs',
      'ra = 1:12:43.2, dec = +1:12:43 :')
print(coord_from_hourangle, '\n')


# Create a coordinate object from pixel values
# Notice that this time we have to use `.from_pixel`
cord_from_pixel = SkyCoord.from_pixel(170, 740, image_wcs)
print('A SkyCord object created with pixel coordinates ra = 170, dec = 740 :')
print(cord_from_pixel, '\n')


# We can also convert the coordinates back to pixel values
print('The pixel coordinates corresponding to',
      '(approximatly) ra = 85.35 deg, dec = -2.38 deg :')
print(cord_from_pixel.to_pixel(image_wcs))


Typically, the coordinates of the object you're interested in will have to be read in from another file (even though they are "hard coded" in the example above, which is considered bad programming practice). One of the attractive features of SkyCord is that it can handle a wide range of input units depending on what the format of your input file is.

Also, notice that in the image a few cells above, the y-axis starts at the top. Be careful about assuming the origin of the axis used to reference an image.

## Performing Photometry <a class="anchor" id="performing-photometry"></a>


The `photutils` package provides a few different aperture shapes (circle, oval, rectangular, annulus, etc.). We will be using the circular shaped aperture. For convenience, all of the `photutils` apertures come in pairs - one for pixel coordinates, and one for sky coordinates. For example, if you want a circular aperture in pixel coordinates, you would use the `CircularAperture` object. If you were working in sky coordinates, you would use the `SkyCircularAperture` object. 

Once you have selected your aperature, you can use the `aperture_photometry` function to perfrom apererture photometry. As an example, let's perform photometry on the bright star in the lower left corner of the example image using an aperture with a 10 arcsecond radius.

For reference, here is what the aperture looks like when drawn on the image (understanding the code in the following cell is not important):


In [ ]:
import astropy
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
import numpy as np
from photutils import SkyCircularAperture
from astropy.coordinates import SkyCoord

hdu_list = astropy.io.fits.open("example.fits")
wcs = astropy.wcs.WCS(hdu_list[0])

coord = SkyCoord(85.3522, -2.37612, unit=('deg'))
aperture = SkyCircularAperture(coord, 10 * astropy.units.arcsec)
pixel_radius = np.sqrt(aperture.to_pixel(wcs).area() / np.pi)

fig,ax = plt.subplots(1)
ax.imshow(hdu_list[0].data, cmap='gray')
circ = Circle((170, 740), radius=pixel_radius,
              facecolor='None', edgecolor='r', lw=1)

ax.add_patch(circ)
plt.show()


Before starting we need to look up the following information:

- The object is located at (85.3522, -2.37612) degrees
- The obect has a redshift of .5 (made up number for demonstration purposes)
- For the survey that took this image, 1 photon count is equal to 1.5 ergs / cm$^2$ (also a made up number)

This information tells us the location of the object in the image, how far away it is (we can calculate this from the redshift), and how to convert our results into units of energy. 

First, let's determine the total photon counts within the aperture:

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u
from numpy import pi
from photutils import aperture_photometry, SkyCircularAperture

hdu_list = fits.open("example.fits")
primary_hdu = hdu_list[0]

# The radius should be dimensionful
aperture_radius = 10 * u.arcsec

# We create an aperture using the coordinates and radius
aperture_coord = SkyCoord(85.3522, -2.37612, unit=('deg'))
aperture = SkyCircularAperture(aperture_coord, aperture_radius)

# We perfrom photometry. This will return results as a table
photometry_results = aperture_photometry(primary_hdu, aperture)
photometry_results

We can now convert the sum of pixel values within the aperture to units of energy by using the conversion factor. We can access a column in the table by name, and a row by its index number. Note that the `id` number above is NOT the index number, since Python starts indices from 0.

In [ ]:
# The conversion factor does not have to be dimensionful
# but doing so helps keep track of the units
conv_factor = 1.5 * u.erg / (u.s * u.cm * u.cm * u.AA)
photometry_energy = photometry_results['aperture_sum'][0] * conv_factor
print(photometry_energy)

The total energy output of the star is thus given by:

In [ ]:
# This converts redshift to distance
redshift = .5
ldist = cosmo.luminosity_distance(redshift)

# We get the value of the distance in centimeters
ldist_cm = ldist.cgs

# We find the luminosity by multiplying by the total area
total_energy = photometry_energy * 4 * pi * (ldist_cm ** 2) 
print(total_energy)

Notice that our final results are in units of energy per wavelength per time.

For simplicity, here is the above code simplified into a single cell

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.cosmology import WMAP9 as cosmo
from astropy import units as u
from numpy import pi
from photutils import aperture_photometry, SkyCircularAperture

hdu_list = fits.open("example.fits")
primary_hdu = hdu_list[0]

# Create the aperture
aperture_radius = 10 * u.arcsec
aperture_coord = SkyCoord(85.3522, -2.37612, unit=('deg'))
aperture = SkyCircularAperture(aperture_coord, aperture_radius)

# Perform aperture photometry
photometry_results = aperture_photometry(primary_hdu, aperture)
conv_factor = 1.5 * u.erg / (u.s * u.cm * u.cm * u.AA)
photometry_energy = photometry_results['aperture_sum'][0] * conv_factor

print(photometry_energy)

# Convert results to total energy
redshift = .5
ldist_cm = cosmo.luminosity_distance(redshift).cgs
total_energy = photometry_energy * 4 * pi * (ldist_cm ** 2) 
print(total_energy)